In [ ]:
%gui qt
from stl import mesh
from mayavi import mlab
import numpy as np
import matplotlib.pyplot as plt

from kifmm_py import KiFmm

In [ ]:
def read_stl_triangle_mesh_vertices(filepath, dtype=np.float32):
    """Read STL into Fortran ordered NumPy array"""
    m = mesh.Mesh.from_file(filepath).vectors

    faces = m.reshape(-1, 3)
    faces = np.arange(faces.shape[0]).reshape(-1, 3)  # Assuming each face is a triangle

    x = m[:, :, 0].flatten()
    y = m[:, :, 1].flatten()
    z = m[:, :, 2].flatten()

    # Return as a single Fortran order array
    n = len(x)
    result = np.zeros(3 * n).astype(dtype)
    result[:n] = x
    result[n : 2 * n] = y
    result[2 * n :] = z
    return (np.reshape(result, [n, 3], order="F").astype(dtype), faces)

# Plot Figure

In [ ]:
(sources, faces) = read_stl_triangle_mesh_vertices("example.stl")

x = sources[:, 0]
y = sources[:, 1]
z = sources[:, 2]

In [ ]:
mlab.view(azimuth=40, elevation=70, distance="auto", focalpoint="auto")
fig = mlab.figure(
    size=(960, 1080), bgcolor=(1, 1, 1)
)  # This sets the window size for rendering
plot = mlab.triangular_mesh(
    x, y, z, faces, color=(0.5, 0.5, 0.5), representation="surface", figure=fig
)
mlab.show()

# Run FMM

In [ ]:
dim = 3
dtype = np.float32

# Set FMM Parameters
expansion_order = 6
n_vec = 1
n_crit = 150
n_sources = sources.shape[0]
n_targets = sources.shape[0]
kernel = "laplace"  # Or 'helmholtz'
field_translation = "blas"  # Or 'fft'
kernel_eval_type = "eval"  # For potential graidents, or 'eval' for potentials only

# Set Random charges
charges = np.reshape(
    np.random.rand(n_sources * n_vec), (n_sources, n_vec), order="F"
).astype(dtype)

fmm = KiFmm(
    expansion_order,
    sources,
    sources,
    charges,
    n_crit,
    kernel_eval_type,
    kernel,
    field_translation,
    svd_threshold=1e-7,  # Optional, required for 'blas' based field translations
)

fmm.evaluate()

### Data has been re-ordered by global sort during tree construction

In [ ]:
potentials = fmm.all_potentials()

reordered = np.zeros_like(potentials)
for i, j in enumerate(fmm.target_global_indices):
    reordered[j] = potentials[i]

### Plot Solution

In [ ]:
mlab.view(azimuth=40, elevation=70, distance="auto", focalpoint="auto")
fig = mlab.figure(
    size=(960, 1080), bgcolor=(1, 1, 1)
)  # This sets the window size for rendering
solution = mlab.triangular_mesh(
    x, y, z, faces, scalars=np.log(reordered).flatten(), representation="surface"
)